In [1]:
from ratelimit import limits 
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)
from glob import glob
import json
from pprint import pprint
import numpy as np
import os


In [2]:
dynamic_malware_dataset_path ="c:\hackathon\dataset\Dynamic_Analysis_Data_Part*\Malware\\*\\*"
static_malware_dataset_path ="c:\hackathon\dataset\Static_Analysis_Data\Malware\\*\\*"
dynamic_benign_dataset_path ="c:\hackathon\dataset\Dynamic_Analysis_Data_Part*\Benign\\*"
static_benign_dataset_path ="c:\hackathon\dataset\Static_Analysis_Data\Benign\\*\\*"

In [3]:
dynamic_malware = glob(dynamic_malware_dataset_path)
static_malware = glob(static_malware_dataset_path)
dynamic_benign = glob(dynamic_benign_dataset_path)
static_benign = glob(static_benign_dataset_path) 

all_files = dynamic_benign + dynamic_malware

In [4]:
rows = []
for i, path in zip(range(19957),all_files):
    row = {}
    row["File"] = path 
    row["Category"] = path.split("\\")[4]
    row["SubCategory"] =  path.split("\\")[5] if row["Category"] == "Malware" else "Benign"
    rows.append(row)

In [5]:
df = pd.DataFrame(rows)
df.groupby(["Category","SubCategory"]).count()

File
Category SubCategory           
Benign   Benign            4994
Malware  Backdoor           717
         Trojan             998
         TrojanDownloader   766
         TrojanDropper     1000
         Virus              566
         Worm               914

In [6]:
df["File"][0]

'c:\\hackathon\\dataset\\Dynamic_Analysis_Data_Part1\\Benign\\0028674d11b4995333b419ca593b6596ed482c28fba06b9349718a838a43618e.json'

In [7]:
df.groupby(["Category"]).count()

,File,SubCategory
Category,,
Benign,4994,4994
Malware,4961,4961


In [8]:
def load_file(file_path):
    with open(file_path) as file_obj:
        json_obj = json.loads(file_obj.read())
    return(json_obj)

In [9]:
def extract_signature(file_path):
    js = load_file(file_path)
    signatures = js["signatures"]
    return signatures
def extract_mark_count(file_path):

    print(file_path)
    max_mark = -1
    signatures = extract_signature(file_path)
    for sign in signatures:
        if sign["markcount"] > max_mark:
            max_mark = sign["markcount"]       
    return max_mark

def extract_virus_total_mark_count(file_path):
    signatures = extract_signature(file_path)
    
    for sign in signatures:
        if "VirusTotal" in sign["description"]:
            desc = sign["markcount"]
            break
    else:
        desc = 0
        
    return desc

In [10]:
# for i,file_path in zip(range(10),df["File"]):
#     pprint(extract_virus_total_description(file_path))
    

In [11]:
if os.path.isfile(r'C:\Users\Akash\mark_count.csv'):
    df = pd.read_csv(r'C:\Users\Akash\mark_count.csv')
else:
    extract_mark_count = np.vectorize(extract_mark_count)
    mark_counts = extract_mark_count(df["File"])
    extract_virus_total_mark_count = np.vectorize(extract_virus_total_mark_count)
    virus_total_mark_counts = extract_virus_total_mark_count(df["File"])

    df["Mark Count"] = mark_counts
    df["Virus Total Count"] = virus_total_mark_counts

In [12]:

df

,Unnamed: 0,File,Category,SubCategory,Mark Count,Virus Total Count
0,0,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Benign,Benign,1,0
1,1,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Benign,Benign,-1,0
2,2,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Benign,Benign,1,0
3,3,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Benign,Benign,1,0
4,4,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Benign,Benign,1,0
...,...,...,...,...,...,...
9950,9950,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,48,48
9951,9951,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,41,41
9952,9952,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,62,62
9953,9953,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,49,49


In [14]:
anomaly_malwares = df[(df["Category"] == "Malware") & (df["Virus Total Count"] == 0)]

In [65]:
def check_string_presence(file_path, string=""):
    if string in open(file_path).read():
        return 1
    else:
        return 0
check_string_presence = np.vectorize(check_string_presence)
anomaly_malwares["shit_over_string_presence"] = check_string_presence(anomaly_malwares["File"])
anomaly_malwares["icmp_traffic_presence"] = check_string_presence(anomaly_malwares["File"],string="Generates some ICMP traffic")
anomaly_malwares["InternetOpenURLA"] = check_string_presence(anomaly_malwares["File"],string="InternetOpenURLA")
anomaly_malwares["high_port_presence"] = check_string_presence(anomaly_malwares["File"],"Communication to multiple IPs on high port numbers")

C:\Users\Akash\anaconda3\envs\iitk\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Akash\anaconda3\envs\iitk\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Akash\anaconda3\envs\iitk\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [66]:
checks = check_string_presence(df[df["Category"] == "Benign"]["File"])

In [53]:
anomaly_malwares.groupby("SubCategory")["high_port_presence"].sum()


NameError: name 'meh' is not defined

In [ ]:
df["Category"].value_counts()

In [ ]:
df[df["Mark Count"] == 5610]["File"].tolist()

In [25]:
df.to_csv(r'C:\Users\Akash\mark_count.csv')

In [26]:
df

,File,Category,SubCategory,Mark Count,Virus Total Count
0,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Benign,Benign,1,0
1,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Benign,Benign,-1,0
2,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Benign,Benign,1,0
3,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Benign,Benign,1,0
4,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Benign,Benign,1,0
...,...,...,...,...,...
9950,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,48,48
9951,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,41,41
9952,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,62,62
9953,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,49,49


In [44]:
descriptions = {}
for i,file_name in enumerate(anomaly_malwares["File"]):
    signatures = extract_signature(file_name)
    for sign in signatures:
        desc_count = descriptions.get(sign["description"],0)
        desc_count+=1
        descriptions[sign["description"]] = desc_count
    if i%10 == 0:
        print(i)


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280


In [45]:
desc_df = pd.DataFrame([{"Description":i,"Count":k} for i,k in descriptions.items()])

In [46]:
desc_df.sort_values(by="Count",ascending=False).to_csv("Descriptions_for_Anomalies.csv")

In [41]:
desc_df["Description"] == "Communication to multiple IPs on high port numbers possibly indicative of a peer-to-peer (P2P) or non-standard command and control protocol"
    file

0      False
1      False
2      False
3      False
4      False
       ...  
176    False
177    False
178    False
179    False
180    False
Name: Description, Length: 181, dtype: bool

desc_df.sort_values(by="Count",ascending=False)

In [64]:
pd.read_csv("Descriptions_for_Malware.csv")

,Unnamed: 0,Description,Count
0,3,Generates some ICMP traffic,4891
1,1,Communication to multiple IPs on high port num...,3260
2,0,One or more processes crashed,2319
3,26,"Checks amount of memory in system, this can be...",2232
4,2,Allocates read-write-execute memory (usually t...,1999
...,...,...,...
150,126,Writes a potential ransom message to disk,1
151,127,Overwrites 52 files indicative of destructive ...,1
152,129,Creates (office) documents on the filesystem,1
153,131,Modifies the Firefox configuration file,1


In [57]:
pd.read_csv("Descriptions_for_Benign.csv")["Count"].std()

284.942258590244

In [58]:
So we have our features : Presence of virus total count, presence of ICMP traffic, presence of comms with high port(weak)

SyntaxError: invalid syntax (<ipython-input-58-c913ad771ae8>, line 1)

In [63]:
anomaly_malwares[anomaly_malwares["high_port_presence"] == 0]

,Unnamed: 0,File,Category,SubCategory,Mark Count,Virus Total Count,shit_over_string_presence,icmp_traffic_presence,high_port_presence
5286,5286,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Backdoor,3,0,0,1,0
5341,5341,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Backdoor,7,0,0,1,0
5618,5618,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Backdoor,3,0,0,1,0
5729,5729,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Trojan,11,0,0,1,0
5731,5731,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Trojan,16,0,0,1,0
...,...,...,...,...,...,...,...,...,...
9373,9373,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,2,0,0,1,0
9471,9471,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,2,0,0,1,0
9679,9679,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,9,0,0,1,0
9714,9714,c:\hackathon\dataset\Dynamic_Analysis_Data_Par...,Malware,Worm,4,0,0,1,0
